In [1]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from keras.layers.core import Dense, Permute, Reshape, Flatten 
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical

np.random.seed(42)

Using TensorFlow backend.


In [2]:
drive.mount('/content/Drive/',force_remount =  True)

Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/Final Data/"

In [0]:
with open(dirpath + 'ca2500_noise_30_mfcc.pkl', 'rb') as f:
  ca_noise_30= pickle.load(f)
  
with open(dirpath + 'de2500_noise_30_mfcc.pkl', 'rb') as f:
  de_noise_30 = pickle.load(f)
 
with open(dirpath + 'it2500_noise_30_mfcc.pkl', 'rb') as f:
  it_noise_30= pickle.load(f)
  
with open(dirpath + 'en2500_noise_30_mfcc.pkl', 'rb') as f:
  en_noise_30 = pickle.load(f)
 
with open(dirpath + 'fr2500_noise_30_mfcc.pkl', 'rb') as f:
  fr_noise_30 = pickle.load(f)
 


In [0]:
data_30 = np.concatenate((ca_noise_30, de_noise_30, it_noise_30, en_noise_30, fr_noise_30), axis = 0)

In [0]:
del(ca_noise_30)
del(de_noise_30)
del(it_noise_30)
del(en_noise_30)
del(fr_noise_30)

In [0]:
np.random.shuffle(data_30)

In [8]:

labels_30 =[]
for i in range(len(data_30)):

      labels_30.append(data_30[i][1])  

labels_30 = np.array(labels_30)
labels_30

classnames, indices = np.unique(labels_30, return_inverse=True)
print(indices)
print(classnames)


final_labels_30 = indices
final_lables30_cat = to_categorical(final_labels_30)

[0 2 1 ... 4 0 4]
['Catalan' 'English' 'French' 'German' 'Italian']


In [9]:
final_X_30 =[]
for i in range(len(data_30)):

      final_X_30.append(data_30[i][0])  

final_X_30 = np.array(final_X_30)
final_X_30[0].shape

(972, 40)

In [0]:
del(data_30)

In [0]:
train_x = np.array(final_X_30[:8000])
cv_x = np.array(final_X_30[8000:10000])
test_x = np.array(final_X_30[10000:])
train_y = np.array(final_lables30_cat[:8000])
cv_y = np.array(final_lables30_cat[8000:10000])
test_y = np.array(final_lables30_cat[10000:])

In [0]:
# final_data_tr

In [19]:
model = Sequential()
weight_decay = 0.001

model.add(Convolution2D(32, kernel_size=(4,4), strides=(1,1), activation="relu", padding="same", kernel_regularizer=l2(weight_decay), input_shape=(972,40,1)))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(64, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))



model.add(Convolution2D(128, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(256, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

#model.add(Convolution2D(512, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
#model.add(BatchNormalization())

#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


#model.add(Convolution2D(512, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
#model.add(BatchNormalization())

#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

print(model.layers[-1].output_shape)

#model.add(Permute((2,1,3)))

print(model.layers[-1].output_shape)

bs, x, y, c = model.layers[-1].output_shape

model.add(Reshape([x,y*c]))

print(model.layers[-1].output_shape)

#model.add(LSTM(256, activation="tanh", return_sequences=True))

model.add(LSTM(512, activation="tanh", return_sequences=True))

model.add(Flatten())

# model.add(Dense(256, activation="relu"))

# model.add(Dropout(rate=0.7))

model.add(Dense(5, activation="softmax"))

(None, 61, 3, 256)
(None, 61, 3, 256)
(None, 61, 768)


In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 972, 40, 32)       544       
_________________________________________________________________
batch_normalization_7 (Batch (None, 972, 40, 32)       128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 486, 20, 32)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 486, 20, 64)       32832     
_________________________________________________________________
batch_normalization_8 (Batch (None, 486, 20, 64)       256       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 243, 10, 64)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 243, 10, 128)      131200    
__________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.00005),
             metrics=["accuracy"])

In [0]:
model.fit(train_x.reshape(-1,972,40,1), train_y,
          batch_size=200,
          epochs=100,
          verbose=1,
          validation_data=(cv_x.reshape(-1,972,40,1),cv_y)
         )
          

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 36s 5ms/step - loss: 1.2153 - acc: 0.6370 - val_loss: 1.0540 - val_acc: 0.6875
Epoch 2/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.9406 - acc: 0.7566 - val_loss: 0.9862 - val_acc: 0.7245
Epoch 3/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.8393 - acc: 0.8010 - val_loss: 0.9175 - val_acc: 0.7585
Epoch 4/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.7655 - acc: 0.8324 - val_loss: 0.8929 - val_acc: 0.7710
Epoch 5/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.7015 - acc: 0.8561 - val_loss: 0.8501 - val_acc: 0.7775
Epoch 6/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.6411 - acc: 0.8823 - val_loss: 0.8519 - val_acc: 0.7855
Epoch 7/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.5639 - acc: 0.9125 - val_loss: 0.8379 - val_acc: 0.7895


In [0]:
   # # from keras.models import load_model

# # model.save(dirpath + 'my_model.h5')  # creates a HDF5 file 'my_model.h5'
# # del model  # deletes the existing model

# # # returns a compiled model
# # # identical to the previous one
# model = load_model('Drive/My Drive/DL project/my_model.h5')

In [0]:
y_pred = model.predict(test_x.reshape(-1,972,40,1))
accuracy = accuracy_score(test_y.argmax(axis=1), y_pred.argmax(axis=1))
matrix = confusion_matrix(test_y.argmax(axis=1), y_pred.argmax(axis=1))

In [0]:
print(accuracy)
print(matrix)

In [0]:
matrix

In [0]:
np.unique(ty, return_counts=True)

(array([0, 1, 2, 3, 4]), array([1600, 1617, 1603, 1599, 1581]))

In [0]:
np.unique(cvy, return_counts=True)
  

(array([0, 1, 2, 3, 4]), array([400, 383, 397, 401, 419]))